# Feature Enginnering
- "Peak Signal Characterisation" Spalten werden auf "Modality","Tail" und "Shoulder" reduziert
- Polymer wird aus der "Name of Original DSC Measurement" Spalte etrahiert

In [2]:
import pandas as pd
import re

In [3]:
file_path_data = "data/03_Data_2nd_Heating.csv"
df_data = pd.read_csv(file_path_data)
df_data

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,50.00,118.90,107.6,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,50.00,118.20,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,50.00,119.00,107.8,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,205.46,254.29,239.9,0,0,x,s,0.0,0,0.0,0,0,Third EBT
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,201.99,251.99,239.9,0,0,x,s,0.0,0,0.0,0,0,Third EBT
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,0,0,x,s,0.0,0,0.0,0,0,Third EBT
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,0,0,x,s,0.0,0,0.0,0,0,Third EBT


In [4]:
# Extracting distinct values from columns related to "Peak Signal Characterisation"
peak_signal_columns = [col for col in df_data.columns if "Peak Signal Characterisation" in col]
distinct_values = {col: df_data[col].unique().tolist() for col in peak_signal_columns}

# Output the distinct values
for column, values in distinct_values.items():
    print(f"{column}: {values}")

Peak Signal Characterisation_Bimodal: ['0', 'x', '1>2', '1=2', '1<2']
Peak Signal Characterisation_Sharp Monodal: ['x', '0']
Peak Signal Characterisation_Wide Monodal: ['0', 'x']
Peak Signal Characterisation_Left Tail: ['l', 'x', 's']
Peak Signal Characterisation_Right Tail: [0.0]
Peak Signal Characterisation_Left Shoulder: [0]
Peak Signal Characterisation_Right Shoulder: [0.0]
Peak Signal Characterisation: ['0', '1<2']
Peak Signal Characterisation_Quasibimodal: ['0', 'x']


In [5]:
def sort_columns(df, last_columnname='Aging Level'):
    """
    Sortiert die Spalten eines DataFrames in folgender Reihenfolge:
    1. Spalten, die nicht mit 'T' oder 'Peak Signal' beginnen und nicht die letzte Spalte sind.
    2. Spalten, die mit 'T' beginnen.
    3. Spalten, die mit 'Peak Signal' beginnen.
    4. Die letzte Spalte (standardmäßig 'Aging Level').

    Parameters:
        df (pd.DataFrame): Der zu sortierende DataFrame.
        last_columnname (str): Name der letzten Spalte (Standard: 'Aging Level').

    Returns:
        pd.DataFrame: DataFrame mit neu sortierten Spalten.
    """
    columns = df.columns
    characterisation=["Modality", "Tail", "Shoulder"]

    # Listen für die Sortierung
    first_columns = [col for col in columns if not col.startswith(('T','H', 'Peak Signal')) and col != last_columnname and col not in characterisation]
    t_columns = [col for col in columns if col.startswith('T') and col not in characterisation]
    h_columns = [col for col in columns if col.startswith('H')]
    peak_columns = [col for col in columns if col.startswith('Peak Signal')]
    last_column = [last_columnname] if last_columnname in columns else []

    # Neu sortieren: erst 'first_columns', dann 't_columns', dann 'peak_columns', dann 'last_column'
    sorted_columns = first_columns + t_columns + h_columns + peak_columns + characterisation +  last_column

    # DataFrame mit neu sortierten Spalten zurückgeben
    return df[sorted_columns]

### a) Feature Consolidation  - Peak Signal Characterisation

In [7]:
# Funktion zur Festlegung der Peak Signal Characterisation
def identify_characterisation_row(row):
    
    # Helper-Funktion, um zu prüfen, ob ein Wert als "nicht 0" gilt
    def is_not_zero(value):
        return str(value) != '0' and value != 0 and value != 0.0

    # Modality bestimmen
    modality_columns = {
        'bimodal': is_not_zero(row['Peak Signal Characterisation_Bimodal']),
        'sharp': is_not_zero(row['Peak Signal Characterisation_Sharp Monodal']),
        'wide': is_not_zero(row['Peak Signal Characterisation_Wide Monodal']),
        'quasi': is_not_zero(row['Peak Signal Characterisation_Quasibimodal'])
    }
    
     # Wenn kein Wert einen Eintrag hat oder mehrere Werte vorliegen, dann 'undefined'
    if sum(modality_columns.values()) == 1:
        modality = [key for key, value in modality_columns.items() if value][0]
    else:
        modality = 'undefined'

    # Tail bestimmen
    if is_not_zero(row['Peak Signal Characterisation_Left Tail']):
        tail = 'left'
    elif is_not_zero(row['Peak Signal Characterisation_Right Tail']):
        tail = 'right'
    else:
        tail = 'undefined'

    # Shoulder bestimmen
    if is_not_zero(row['Peak Signal Characterisation_Left Shoulder']):
        shoulder = 'left'
    elif is_not_zero(row['Peak Signal Characterisation_Right Shoulder']):
        shoulder = 'right'
    else:
        shoulder = 'undefined'

    return pd.Series([modality, tail, shoulder], index=['Modality', 'Tail', 'Shoulder'])



# Funktion zur Transformation des gesamten DataFrames
def transform_peak_signal_characteristics(df):
    # Transformation anwenden
    df[['Modality', 'Tail', 'Shoulder']] = df.apply(identify_characterisation_row, axis=1)
    
    # Alle Spalten mit "Peak Signal Characterisation" löschen
    columns_to_drop = [col for col in df.columns if 'Peak Signal Characterisation' in col]
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

In [8]:
# Transformation anwenden
df_transformed = transform_peak_signal_characteristics(df_data)
df_transformed

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Aging Level,Modality,Tail,Shoulder
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,Pristine LDPE and PET,sharp,left,undefined
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,Pristine LDPE and PET,sharp,left,undefined
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,50.00,118.90,107.6,Pristine LDPE and PET,sharp,left,undefined
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,50.00,118.20,107.7,Pristine LDPE and PET,sharp,left,undefined
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,50.00,119.00,107.8,Pristine LDPE and PET,sharp,left,undefined
...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,205.46,254.29,239.9,Third EBT,wide,left,undefined
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,201.99,251.99,239.9,Third EBT,wide,left,undefined
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,Third EBT,wide,left,undefined
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,Third EBT,wide,left,undefined


In [9]:
# Häufigkeiten der Werte für jede Spalte berechnen und einzeln anzeigen
print(f"{df_transformed['Modality'].value_counts()}\n")
print(f"{df_transformed['Tail'].value_counts()}\n")
print(f"{df_transformed['Shoulder'].value_counts()}\n")

Modality
bimodal    155
sharp       79
wide        56
quasi        2
Name: count, dtype: int64

Tail
left    292
Name: count, dtype: int64

Shoulder
undefined    292
Name: count, dtype: int64



### b) Feature Generation - Hm (y-Wert zu der Temperatur Tm) 

In [11]:
df_experiments = pd.read_csv("data/Experiments_2nd_Heating.csv", index_col = 0)
df_experiments

,0-PE-10,0-PE-11,0-PE-12,0-PE-13,0-PE-14,0-PE-15,0-PE-16,0-PE-17,0-PE-18,0-PE-19,...,UV-PET-59,UV-PET-6,UV-PET-60,UV-PET-7,UV-PET-76,UV-PET-77,UV-PET-78,UV-PET-79,UV-PET-8,UV-PET-9
Temp./°C,,,,,,,,,,,,,,,,,,,,,
-25,3.94766,3.91972,4.05790,4.13289,3.82275,4.00157,4.18087,3.91814,4.97739,4.22145,...,3.95683,1.88544,4.02637,2.17032,2.98046,2.88987,3.46959,3.40215,1.70131,1.87894
-24,3.95250,3.92429,4.06407,4.14106,3.82973,3.99952,4.19073,3.92699,4.99383,4.22974,...,3.97837,1.88835,4.03066,2.16730,2.98969,2.89599,3.47728,3.39419,1.70440,1.86790
-23,3.95307,3.92736,4.06864,4.15010,3.83740,4.01068,4.19749,3.93712,4.98818,4.23567,...,3.97614,1.88846,4.05506,2.17359,2.99713,2.90810,3.48758,3.40015,1.71841,1.87305
-22,3.96356,3.93813,4.07811,4.15815,3.84421,4.01827,4.20877,3.94424,5.00459,4.25181,...,3.99298,1.87911,4.05299,2.16390,3.00757,2.92446,3.49876,3.42468,1.70963,1.87415
-21,3.97627,3.94550,4.09258,4.17234,3.85675,4.03683,4.21752,3.95309,5.01714,4.25870,...,4.01335,1.88492,4.05911,2.17059,3.01472,2.91728,3.50415,3.43563,1.71217,1.88214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,6.77143,6.85277,7.17616,7.23334,6.77260,7.11991,7.26021,7.18403,8.81364,7.51063,...,7.44245,2.85871,7.61240,3.09266,5.56269,5.37605,6.44393,6.29337,2.52425,2.69898
271,6.78967,6.87615,7.19767,7.24583,6.79039,7.13805,7.27224,7.20013,8.83735,7.52670,...,7.45806,2.86621,7.63346,3.09930,5.57293,5.37883,6.45563,6.30691,2.53141,2.70881
272,6.80702,6.89665,7.21424,7.25314,6.81041,7.15291,7.27928,7.21753,8.85928,7.54118,...,7.47667,2.87413,7.64855,3.10573,5.58632,5.39402,6.47043,6.31307,2.53573,2.71216


In [12]:
# Spalte "Tm in °C" in Integer (gerundet) konvertieren
df_transformed["Tm"] = df_transformed["Tm in °C"].round().astype(int)
df_transformed

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Aging Level,Modality,Tail,Shoulder,Tm
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,Pristine LDPE and PET,sharp,left,undefined,108
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,Pristine LDPE and PET,sharp,left,undefined,108
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,50.00,118.90,107.6,Pristine LDPE and PET,sharp,left,undefined,108
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,50.00,118.20,107.7,Pristine LDPE and PET,sharp,left,undefined,108
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,50.00,119.00,107.8,Pristine LDPE and PET,sharp,left,undefined,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,205.46,254.29,239.9,Third EBT,wide,left,undefined,240
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,201.99,251.99,239.9,Third EBT,wide,left,undefined,240
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,Third EBT,wide,left,undefined,240
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,Third EBT,wide,left,undefined,239


In [13]:
count = 0

# Leere Spalte "Hm" initialisieren
df_transformed["Hm"] = None

# Iterative Schleife, um Werte zuzuweisen
for i, row in df_transformed.iterrows():
    temp = row["Tm"]  # Temperatur
    measurement_name = row["Name of Original DSC Measurement"]  # Name der Messung
    
    # Überprüfung der Index- und Spaltenexistenz
    temp_exists = temp in df_experiments.index
    column_exists = measurement_name in df_experiments.columns

    if temp_exists and column_exists:
        # Wert zuweisen
        df_transformed.at[i, "Hm"] = df_experiments.at[temp, measurement_name]
    else:
        print(f"{measurement_name} fehlt in den Messwerten")
        count+=1

print(f"\nEs fehlen ingesamt {count} Messwerte")

0-PET-26 fehlt in den Messwerten
UV-PE-63 fehlt in den Messwerten
UV-PET-61 fehlt in den Messwerten
UV-PET-62 fehlt in den Messwerten
UV-PET-63 fehlt in den Messwerten
UV-PET-64 fehlt in den Messwerten
UV-PET-65 fehlt in den Messwerten
UV-PET-66 fehlt in den Messwerten
UV-PET-67 fehlt in den Messwerten
UV-PET-68 fehlt in den Messwerten
UV-PET-69 fehlt in den Messwerten
UV-PET-70 fehlt in den Messwerten
UV-PET-71 fehlt in den Messwerten
UV-PET-72 fehlt in den Messwerten
UV-PET-73 fehlt in den Messwerten
UV-PET-74 fehlt in den Messwerten
UV-PET-75 fehlt in den Messwerten

Es fehlen ingesamt 17 Messwerte


In [14]:
# Entfernen der Zeilen mit None/NaN-Werten in der Spalte "Hm"
df_transformed = df_transformed.dropna(subset=["Hm"])
df_transformed

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Aging Level,Modality,Tail,Shoulder,Tm,Hm
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,Pristine LDPE and PET,sharp,left,undefined,108,8.79001
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,Pristine LDPE and PET,sharp,left,undefined,108,8.76131
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,50.00,118.90,107.6,Pristine LDPE and PET,sharp,left,undefined,108,8.71668
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,50.00,118.20,107.7,Pristine LDPE and PET,sharp,left,undefined,108,8.40655
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,50.00,119.00,107.8,Pristine LDPE and PET,sharp,left,undefined,108,7.96444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,205.46,254.29,239.9,Third EBT,wide,left,undefined,240,6.79035
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,201.99,251.99,239.9,Third EBT,wide,left,undefined,240,6.0888
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,Third EBT,wide,left,undefined,240,6.31048
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,Third EBT,wide,left,undefined,239,6.60616


### c) Feature Extraction - Polymer

In [16]:
df_transformed["Name of Original DSC Measurement"]

0         0-PE-6
1         0-PE-7
2         0-PE-8
3         0-PE-9
4        0-PE-10
         ...    
287    eb-PET-62
288    eb-PET-63
289    eb-PET-64
290    eb-PET-65
291    eb-PET-66
Name: Name of Original DSC Measurement, Length: 275, dtype: object

In [17]:
df_transformed['Polymer'] = df_transformed['Name of Original DSC Measurement'].apply(
    lambda x: re.search(r'-(.*?)-', x).group(1) if re.search(r'-(.*?)-', x) else None
)

C:\Users\konra\AppData\Local\Temp\ipykernel_9612\2044552630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transformed['Polymer'] = df_transformed['Name of Original DSC Measurement'].apply(


In [18]:
df_transformed['Polymer'].value_counts()

Polymer
PET    138
PE     137
Name: count, dtype: int64

In [19]:
df_transformed = sort_columns(df_transformed)
df_transformed

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,Polymer,TStart in °C,TEnd in °C,Tm in °C,Tm,Hm,Modality,Tail,Shoulder,Aging Level
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,PE,50.00,118.38,107.7,108,8.79001,sharp,left,undefined,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,PE,50.00,119.30,107.7,108,8.76131,sharp,left,undefined,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,PE,50.00,118.90,107.6,108,8.71668,sharp,left,undefined,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,PE,50.00,118.20,107.7,108,8.40655,sharp,left,undefined,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,PE,50.00,119.00,107.8,108,7.96444,sharp,left,undefined,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,PET,205.46,254.29,239.9,240,6.79035,wide,left,undefined,Third EBT
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,PET,201.99,251.99,239.9,240,6.0888,wide,left,undefined,Third EBT
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,PET,202.37,251.33,240.0,240,6.31048,wide,left,undefined,Third EBT
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,PET,198.86,256.10,239.3,239,6.60616,wide,left,undefined,Third EBT


### Export

In [21]:
df_transformed.to_csv("data/050_Data_2st_Heating.csv", index=False)

## Feature-based Splitting nach Polymer

### PET (Polyethylenterephthalat)

In [24]:
# Filtern der Zeilen, in denen Polymer gleich "PET" ist
pet_data = df_transformed[df_transformed['Polymer'] == 'PET']
pet_data

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,Polymer,TStart in °C,TEnd in °C,Tm in °C,Tm,Hm,Modality,Tail,Shoulder,Aging Level
19,0-PET-1_1,0-PET-1,1.44,28.88,2nd Heating,PET,200.00,253.77,230.2,230,2.99577,bimodal,left,undefined,Pristine LDPE and PET
20,0-PET-1_2,0-PET-2,1.35,30.34,2nd Heating,PET,200.79,256.30,230.5,230,2.78461,bimodal,left,undefined,Pristine LDPE and PET
21,0-PET-1_3,0-PET-3,1.39,30.00,2nd Heating,PET,200.00,254.25,229.8,230,3.05761,bimodal,left,undefined,Pristine LDPE and PET
22,0-PET-1_4,0-PET-4,1.28,29.29,2nd Heating,PET,200.00,254.59,230.2,230,2.95315,bimodal,left,undefined,Pristine LDPE and PET
23,0-PET-1_5,0-PET-5,1.34,31.94,2nd Heating,PET,200.00,255.31,229.7,230,2.76109,bimodal,left,undefined,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,3-EBT-PET-4_1,eb-PET-62,1.18,36.62,2nd Heating,PET,205.46,254.29,239.9,240,6.79035,wide,left,undefined,Third EBT
288,3-EBT-PET-4_2,eb-PET-63,1.27,36.95,2nd Heating,PET,201.99,251.99,239.9,240,6.0888,wide,left,undefined,Third EBT
289,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,PET,202.37,251.33,240.0,240,6.31048,wide,left,undefined,Third EBT
290,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,PET,198.86,256.10,239.3,239,6.60616,wide,left,undefined,Third EBT


In [25]:
# Exportieren der gefilterten Daten als CSV
pet_data.to_csv("data/051_Data_2st_Heating_PET.csv", index=False)

### PE (Polyethylen)

In [27]:
# Filtern der Zeilen, in denen Polymer gleich "PE" ist
pe_data = df_transformed[df_transformed['Polymer'] == 'PE']
pe_data

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,Polymer,TStart in °C,TEnd in °C,Tm in °C,Tm,Hm,Modality,Tail,Shoulder,Aging Level
0,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,PE,50.0,118.380,107.7,108,8.79001,sharp,left,undefined,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,PE,50.0,119.300,107.7,108,8.76131,sharp,left,undefined,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,108.10,2nd Heating,PE,50.0,118.900,107.6,108,8.71668,sharp,left,undefined,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,109.40,2nd Heating,PE,50.0,118.200,107.7,108,8.40655,sharp,left,undefined,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,107.10,2nd Heating,PE,50.0,119.000,107.8,108,7.96444,sharp,left,undefined,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,3-EBT-PE-4_1,eb-PE-56,1.34,75.67,2nd Heating,PE,52.5,111.680,93.4,93,6.68234,wide,left,undefined,Third EBT
269,3-EBT-PE-4_2,eb-PE-57,1.38,74.97,2nd Heating,PE,55.0,109.080,93.6,94,6.46357,wide,left,undefined,Third EBT
270,3-EBT-PE-4_3,eb-PE-58,1.35,75.39,2nd Heating,PE,55.0,112.292,93.2,93,6.72744,wide,left,undefined,Third EBT
271,3-EBT-PE-4_4,eb-PE-59,1.36,72.86,2nd Heating,PE,54.6,112.900,93.4,93,6.29116,wide,left,undefined,Third EBT


In [28]:
# Exportieren der gefilterten Daten als CSV
pe_data.to_csv("data/052_Data_2st_Heating_PE.csv", index=False)